In [2]:
import os
os.chdir("/Users/billydodds/Documents/Projects/Transactions_ML")

import pandas as pd
import numpy as np
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split, StratifiedKFold

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.neural_network import MLPClassifier

from components.scripts.load_data import load_data
from components.scripts.MyDecisionTree import MyDecisionTree
from components.scripts.process_data import get_corpora, NLP_distances, run_fold

import multiprocess as multiprocessing # normal multiprocessing doesn't work in Jupyter for some reason

import random

import itertools

# from numpy.ma import MaskedArray
# import sklearn.utils.fixes

# sklearn.utils.fixes.MaskedArray = MaskedArray

# from skopt import BayesSearchCV # didnt fkn work


In [3]:
tr_data = load_data("./components/private_files/")
y = tr_data[~pd.isnull(tr_data.category)].category
X = tr_data[~pd.isnull(tr_data.category)].drop("category", axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

X_train["category"] = y_train
corpora = get_corpora(X_train)
X_train = X_train.drop("category", axis=1)

X_train = NLP_distances(X_train, corpora)
X_test = NLP_distances(X_test, corpora)


<ipython-input-3-4f923a51ee40>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["category"] = y_train


In [27]:
weights = {
    'amount': 5/17,
    'weekday': 5/17,
    'food_desc_dist': 1/17,
    'transport_desc_dist': 1/17,
    'beers_desc_dist': 1/17,
    'shopping_desc_dist': 1/17,
    'wages_desc_dist': 1/17,
    'life/wellbeing_desc_dist': 1/17,
    'transfer_desc_dist': 1/17
}


In [28]:
search_space = {
#         'criterion': ["gini", "entropy"],
#         'criterion': "entropy",
#         'splitter': ["best", "random"],
#         'max_depth': list(range(6, 12, 1)) + [None],
#         'min_samples_split': list(range(6, 12, 1))
#     'n_estimators': np.logspace(0, 4, 5).astype(int)
#     "ccp_alpha": np.linspace(0.0, 0.5, 10)
#     , 'max_features': 'log2'
#         'min_samples_leaf': np.array(range(1, 9, 1))
#         'max_features': list(range(1, 9, 1))
#         'max_features': "log2"
#         'min_impurity_decrease': np.linspace(0.0, 0.05, 10)
#     "class_weight":{"amount":5, "weekday":5}
    }



def my_optimiser_random(X, classifier, search_space:dict, n_iter:int, cv:int):
    best_acc = 0.0
    best_params = None
    
    splitter = StratifiedKFold(n_splits=cv, shuffle=True)
    folds = list(splitter.split(X.drop("category", axis=1), X.category))
    
    for i in range(n_iter):
        model = classifier
        params = {}
        for key, value in search_space.items():
            params[key] = random.choice(value)
        model.set_params(**params)
        
        accuracies = []
        def record(acc):
            accuracies.append(acc)
            
        pool = multiprocessing.Pool(min(multiprocessing.cpu_count(), cv))
        for fold in folds:
            pool.apply_async(run_fold, args=(X, fold, model, False, False, False), callback = record)

        pool.close()
        pool.join()

        acc = np.mean(accuracies)
        if acc > best_acc:
            print(acc)
            best_acc = acc
            best_params = params
            
    return best_acc, best_params


def my_optimiser_brute_force(X, classifier, search_space:dict, cv:int):
    print("starting")
    best_acc = 0.0
    best_params = None
    
    splitter = StratifiedKFold(n_splits=cv, shuffle=True)
    folds = list(splitter.split(X.drop("category", axis=1), X.category))
    
    for vals in itertools.product(*search_space.values()):
        model = classifier
        params = dict(zip(search_space.keys(), vals))
        
        for key, val in {'min_samples_leaf': 1, 'max_depth':None, 'criterion': 'entropy', 'max_features': 2}.items():
            params[key] = val
        params["class_weight"] = weights
        
        model.set_params(**params)
        
        
        accuracies = []
        def record(acc):
            accuracies.append(acc)
        
        acc = run_fold(X, folds[3], model, False, False, False)
        
#         pool = multiprocessing.Pool(min(multiprocessing.cpu_count(), cv))
#         for fold in folds:
#             pool.apply_async(run_fold, args=(X, fold, model, False, False, False), callback = record)

#         pool.close()
#         pool.join()

#         acc = np.mean(accuracies)
        
        print(acc)
        
        if acc > best_acc:
            print(acc)
            best_acc = acc
            best_params = params

    
    return best_acc, best_params
        
X = tr_data[~pd.isnull(tr_data.category)]
best = my_optimiser_brute_force(X, RandomForestClassifier(), search_space, 10)

print(best)

starting


ValueError: Class label amount not present.

In [3]:

X = tr_data[~pd.isnull(tr_data.category)]

n_folds = 10
splitter = StratifiedKFold(n_splits=n_folds, shuffle=True)
folds = list(splitter.split(X.drop("category", axis=1), X.category))

model = DecisionTreeClassifier()


In [11]:

accuracies = []
def record(acc):
    accuracies.append(acc)

pool = multiprocessing.Pool(min(multiprocessing.cpu_count(), n_folds))
for fold in folds:
    pool.apply_async(run_fold, args=(X, fold, model, False, False, False), callback = record)

pool.close()
pool.join()

print(np.mean(accuracies))

0.86


In [33]:


_ = opt.fit(X_train, y_train)

/Users/billydodds/.pyenv/versions/3.8.3/envs/data/lib/python3.8/site-packages/sklearn/utils/deprecation.py:67: FutureWarning: Class MaskedArray is deprecated; MaskedArray is deprecated in version 0.23 and will be removed in version 0.25. Use numpy.ma.MaskedArray instead.
  warnings.warn(msg, category=FutureWarning)


TypeError: object.__init__() takes exactly one argument (the instance to initialize)

In [12]:
dic = {"one":[1, 2, 3], "two":np.array([7, 8]), "three":["spaghetti", "meatballs"]}
for i in itertools.product(*dic.values()):
    print(dict(zip(dic.keys(), i)))

{'one': 1, 'two': 7, 'three': 'spaghetti'}
{'one': 1, 'two': 7, 'three': 'meatballs'}
{'one': 1, 'two': 8, 'three': 'spaghetti'}
{'one': 1, 'two': 8, 'three': 'meatballs'}
{'one': 2, 'two': 7, 'three': 'spaghetti'}
{'one': 2, 'two': 7, 'three': 'meatballs'}
{'one': 2, 'two': 8, 'three': 'spaghetti'}
{'one': 2, 'two': 8, 'three': 'meatballs'}
{'one': 3, 'two': 7, 'three': 'spaghetti'}
{'one': 3, 'two': 7, 'three': 'meatballs'}
{'one': 3, 'two': 8, 'three': 'spaghetti'}
{'one': 3, 'two': 8, 'three': 'meatballs'}


In [15]:
np.logspace(0, 3, 4)

array([   1.,   10.,  100., 1000.])